### Importing CybORG

To use the CybORG environment, it is necessary to import the CybORG class. CybORG stands for **Cyb**er **O**perations **R**esearch **G**ym, so remember to capitalise correctly when importing. 

In [1]:
from CybORG import CybORG

/home/ubuntu/.local/lib/python3.10/site-packages/gym/wrappers/monitoring/video_recorder.py:9: DeprecationWarning: The distutils package is deprecated and slated for removal in Python 3.12. Use setuptools or check PEP 632 for potential alternatives
  import distutils.spawn


path is: /home/ubuntu/CybORG/CybORG/env.py


### Instantiating CybORG

CybORG has to be manually instantiated by calling the class constructor. This must be passed a ScenarioGenerator class, which contains the details of the scenario.
For Challenge 3 we will be using the DroneSwarmScenarioGenerator which creates the correct scenario.

In [2]:
from CybORG.Simulator.Scenarios import DroneSwarmScenarioGenerator

sg = DroneSwarmScenarioGenerator()
cyborg = CybORG(scenario_generator=sg)

env is: sim


/home/ubuntu/.local/lib/python3.10/site-packages/gym/utils/seeding.py:47: DeprecationWarning: WARN: Function `rng.randint(low, [high, size, dtype])` is marked as deprecated and will be removed in the future. Please use `rng.integers(low, [high, size, dtype])` instead.
  deprecation(


### The PettingZoo Interface

Challenge 3 is a multi-agent scenario consisting of several teams of agents. Red team will be attacking the network, Blue team will be defending the network, while Green team represents the network users who will be passing messages to each other via the drone network. For this challenge, the roles of Red and Green will be handled by internal rules-based agents, while your task is to use the external API to train Blue Team.

CybORG supports single and multi-agent reinforcement learning. The single agent API is based on OpenAI gym, while the multi-agent API is based on PettingZoo. Since Challenge 3 is a multi-agent scenario, we will thus use the PettingZooParallelWrapper. We do this by instantiating the wrapper and pass in CybORG as the env parameter.

In [3]:
from CybORG.Agents.Wrappers import PettingZooParallelWrapper

env = PettingZooParallelWrapper(env=cyborg)
obs = env.reset()

print(obs.keys())

Agent Interfaces are: {'red_agent_0': <CybORG.Shared.AgentInterface.AgentInterface object at 0x7f08f6913df0>, 'blue_agent_0': <CybORG.Shared.AgentInterface.AgentInterface object at 0x7f08f67d6dd0>, 'green_agent_0': <CybORG.Shared.AgentInterface.AgentInterface object at 0x7f08f67d6f50>, 'red_agent_1': <CybORG.Shared.AgentInterface.AgentInterface object at 0x7f08f6912410>, 'blue_agent_1': <CybORG.Shared.AgentInterface.AgentInterface object at 0x7f08f67d70a0>, 'green_agent_1': <CybORG.Shared.AgentInterface.AgentInterface object at 0x7f08f67d71f0>, 'red_agent_2': <CybORG.Shared.AgentInterface.AgentInterface object at 0x7f08f6913400>, 'blue_agent_2': <CybORG.Shared.AgentInterface.AgentInterface object at 0x7f08f67d4070>, 'green_agent_2': <CybORG.Shared.AgentInterface.AgentInterface object at 0x7f08f67d47f0>, 'red_agent_3': <CybORG.Shared.AgentInterface.AgentInterface object at 0x7f08f6913370>, 'blue_agent_3': <CybORG.Shared.AgentInterface.AgentInterface object at 0x7f08f67d7370>, 'green_age

/home/ubuntu/CybORG/CybORG/Agents/Wrappers/PettingZooParallelWrapper.py:242: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  new_obs = np.zeros(obs_length, dtype=np.int)


We obtain the initial observation by resetting the environment. This ouputs a dictionary whose keys are the names of the various blue agents and whose values are the corresponding agent's observation. We can see below the observation of 'blue_agent_0'. This is in vector form designed for input into a neural network. See the README page for an explanation of what the values in this vector mean.

In [4]:
print(obs['blue_agent_0'])

[ 1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0 93 83  1 71 55  0  2  0 48  0
  3 34  5  0  4 48 23  0  5  0 89  0  6 57 35  0  7 51 14  0  8 91 10  0
  9 69 11  0 10  0 22  0 11 39  0  0 12 86 96  0 13 31 37  0 14 95 75  0
 15 92 46  0 16 58 23  0 17 77  6  0]


To see the action space for a specific agent, the action_space method is called, passing in the agent name as a string. This returns a Discrete object from OpenAI gym.

In [5]:
action_space = env.action_space('blue_agent_0')
print(action_space)

Discrete(56)


Actions are given to CybORG via the step method. Just like PettingZoo, this is given a dictionary whose keys are the agent names and whose values are the corresponding actions. This function returns the next observation, rewards for the agents, the done signal for each agent and the info dictionary, which is usually empty.

In [6]:
actions = {k:action_space.sample() for k in obs}
obs, reward, done, info = env.step(actions)

print(reward['blue_agent_0'])

0.0


This should be enough to get you started training agents.